# Example of the usage of the Bike Simulation Framework

This file will provide an example on how to use this simulation Framework and all the classes that were implemented for this.
***

| Class | Description  |
|------|------|
|   **BikeStationCDF** | This class will provide all comulative distribution functions for every Bike Stations on the Network.<br> Every station will have at least 7 CDF's which will correspond to every day of the week.|
|**BikeStations**|This class will provide the basic methods and attributes that describe <br> the actions and parameters from which a station can be defined.|
|**BikeRelocationScheme**|This class contains provides a calculation for the relocation weights <br> which correspond to how many bikes will be populated into a station when it reaches a critical level.|
|**RouteAnalysis**|This is a class that interacts with SUMO Simulator to retrive all <br> the objects from the SUMO network as well as all the required metadata|
|**Traci**|Python implementation of SUMO Simulator API's|
|**Geometry**|Distance Calculation, Angle Calculation and other Geometry APIs are provided my this class.|

In [ ]:
from BikeStationCDF import BikeStationCDF as BSCDF
from BikeStations import BikeStationNetwork as BSN
from BikeRelocationScheme import BikeRelocationScheme as BRS

from RouteAnalysis import RouteAnalysis

import traci
from sumolib import checkBinary

from Geometry import GeometryClass

import matplotlib.pyplot as plt
import numpy as np

## SUMO Server Connection

Simulation needs to be running at the time this python script is run. In the case of the current example, the server port is always set 58890 and the connection order is marked as 2 (SUMO requires that each client has its own different connection id).

In [ ]:
traci.init(58890)
traci.setOrder(2)

In [ ]:
myBikeNetwork = BSN()

In [ ]:
RouteAnalysisSUMO = RouteAnalysis()

In [ ]:
AllStationsIds = myBikeNetwork.getAllStationOnNetwork()
RelocationSchemes = BRS(AllStationsIds).getRebalancingWeights()

### BikeInitialStatus

This attribute will hold a list with the number of bikes that are available when simulation starts.

In [ ]:
BikeInitialStatus = [s.availableBikes() for s in list(myBikeNetwork.BikeStationsDict.values())]

### Method: updateStatusOfStations
- **Input: Time -> (float):**
    - The current time of the simulation which goes from 0.0 to 24.0
<br>
- **Input: dayItinerary -> (list(list)):**
    - A list of all the trips that will occur within a single day for every station

#### Details:
This method will take all the trips that are planned within the provided time window and the corresponding **BikeStation Class** APIs will be called to update the state of such Bike Stations.

#### Note:

* This class requires the push/remove bike APIs to be populated with the intended relocation weight in case it is needed, this information comes from BRS().getRebalancingWeights() API *

In [ ]:
def updateStatusOfStations(time, dayItinerary):
    for idx in range(len(dayItinerary)):
        for eventsStation in range(len(dayItinerary[idx])):
            # StationId retrived from trip timestamps of dayItinerary
            StationId = int(dayItinerary[idx][eventsStation][2])
            # Process events only when they're equal to time
            # times are represented in seconds, they need to be multiplied
            # by C_MINUTES_IN_HOUR
            if(int(dayItinerary[idx][eventsStation][0] * C_MINUTES_IN_HOUR) == time):
                # departure or arrival trip type
                tripInstance = dayItinerary[idx][eventsStation]
                
                if(tripInstance[1] == BSN.C_ARRIVALS_IDX):
                    if(myBikeNetwork.getBikeStationObject(
                            StationId).pushBike() is False):
                        # This should never be the case as the
                        # relocation occurs within the scope of
                        # push/removeBike()
                        print(myBikeNetwork.getBikeStationObject(                            
                            StationId).availableBikes())
                        raise

                else:
                    if(myBikeNetwork.getBikeStationObject(
                            StationId).removeBike() is False):
                        # This should never be the case as the
                        # relocation occurs within the scope of
                        # push/removeBike()
                        print(myBikeNetwork.getBikeStationObject(
                            StationId).availableBikes())
                        raise
    

### Method: getNetworkRiskStatus
- **Input: bikeNetwork -> (BikeStationNetwork Object):**
    - This is an object from **BikeStationNetwork Class** which will be used to iterate over all the bike stations status to get the number of available bikes.
<br>
<br>
- **Output: DockRiskStations, BikeRiskStations**
    - The output will be two lists that will contain the number of stations which number of docks is critical and another list that contains the stations which number of bikes is critical.

#### Details:
The output will be two lists that will contain the number of stations which number of docks is critical and another list that contains the stations which number of bikes is critical.

#### Note:

None

In [ ]:
def getNetworkRiskStatus(bikeNetwork):
    # StatusBike Network -> [StationId, NumberofAvailableBikes]
    StatusBikeNetwork = [[int(s), int(bikeNetwork.getBikeStationObject(s).availableBikes())]
        for s in bikeNetwork.BikeStationsDict.keys() ]
    
    data = np.array(StatusBikeNetwork)
    
    # Identify Risk Stations by applying a formula np.max/min +- 1 whcih has calculated empirically
    # to provide a mechanism for Risk Assesment. It's performance is out of the scope of this
    # module for now.
    
    #DockRiskStations -> Stations that have more bikes than docks
    DockRiskStations = data[(data[:,1] >= (np.max(data[:,1]) - 1)) & ((data[:,1]) > C_OPTIMAL_LEVEL)]
    
    #BikeRiskStations -> Stations that have more docks than bikes
    BikeRiskStations = data[(data[:,1] <= (np.min(data[:,1]) + 1)) & ((data[:,1]) < C_OPTIMAL_LEVEL)]

    return DockRiskStations, BikeRiskStations

### Method: upcomingTrips
- **Input: Time -> (float):**
    - The current time of the simulation which goes from 0.0 to 24.0
<br>
- **Input: dayItinerary -> (list(list)):**
    - A list of all the trips that will occur within a single day for every station
<br>
- **Output: dayUpcoming**
    - Returns a list of the trips that will be analyzed within the current time window. The current time window is equal to (0.125) * 60 minutes.

#### Details:
Returns a list of the trips that will be analyzed within the current time window. The current time window is equal to (0.125) * 60 minutes.

#### Note:

None

In [ ]:
def upcomingTrips(time, dayItinerary):
    dayUpcoming = []
    for s in range(len(dayItinerary)):
        UpcomingTrips_bool = ((dayItinerary[s][:,0]*60) >= time) & ((dayItinerary[s][:,0]*60) < (time + (0.125*60)))
        
        # dayUpcoming -> [[list(Trips timestamps)], [dayItinerary bool indexers for location of those trips],
        # dayItineraryIdx]
        if(dayItinerary[s][UpcomingTrips_bool].size > 0):
            dayUpcoming.append([ dayItinerary[s][UpcomingTrips_bool],  UpcomingTrips_bool, s])
            
    return dayUpcoming

### Method: getDistanceBetweenStations
- **Input: RouteAnalysisSUMO -> (RouteAnalysis Object):**
    - Object from RouteAnalysis class that will be used to retrieve the coordinates of the edges to where the bike stations are and which edge centers are required.
<br>
- **Input: stationIdEdge1 -> (String):**
    - Name of the first station.
<br>
- **Input: stationIdEdge1 -> (String):**
    - Name of the seconds station.
<br>
- **Output: Distance between two edges**
    - Returns a **float** value with the distance between the two edges centers. The distance is in meters.

#### Details:
Returns a **float** value with the distance between the two edges centers. The distance is in meters.

#### Note:

None

In [ ]:
def getDistanceBetweenStations(RouteAnalysisSUMO, stationIdEdge1, stationIdEdge2):
    p1 = [float(s) for s in RouteAnalysisSUMO.EdgetoEdgeCenter[stationIdEdge1]]
    p2 = [float(s) for s in RouteAnalysisSUMO.EdgetoEdgeCenter[stationIdEdge2]]
    
    return GeometryClass.getDistance(p1,p2)

### Method: retrieveRiskMeasurements
- **Input: DockRiskStations**
    - A list that will contain the number of stations which number of docks is critical.
- **Input: UpcomingInfo**
    - A list of the trips that will be analyzed within the current time window. The current time window is equal to (0.125) * 60 minutes.
<br>
- **Input: bikeNetwork -> (BikeStationNetwork Object):**
    - This is an object from **BikeStationNetwork Class** which will be used to iterate over all the bike stations status to get the number of available bikes.
<br>
- **Input: RouteAnalysisSUMO -> (RouteAnalysis Object):**
    - Object from RouteAnalysis class that will be used to retrieve the coordinates of the edges to where the bike stations are and which edge centers are required.
<br>
- **Output: NearStations\[Idx\], UpcomingIdx**
    - Returns all the stations that are critical and which current scope trips could be swapped. A distance of no more than 400 meters between stations is consider to suggest these new destination stations. UpcomingIdx corresponds to a boolean list that maps the elements of UpcomingInfo input to the ones that where finally selected.

#### Details:
Returns all the stations that are critical and which current scope trips could be swapped. A distance of no more than 400 meters between stations is consider to suggest these new destination stations. UpcomingIdx corresponds to a boolean list that maps the elements of UpcomingInfo input to the ones that where finally selected.

#### Note:

None

In [ ]:
def retrieveRiskMeasurements(BikeRiskStation, UpcomingInfo, myBikeNetwork, RouteAnalysisSUMO):
    NearStations = []
    UpcomingIdx = []
    for i, StationInfo in enumerate(UpcomingInfo):
        for j, trip in enumerate(StationInfo[0]):
            
            # Only departures will be analyzed by this function
            if((int(trip[1]) != BSN.C_ARRIVALS_IDX) & (int(trip[3]) in myBikeNetwork.getAllStationOnNetwork())):
                distance = \
                    getDistanceBetweenStations(
                        RouteAnalysisSUMO, 
                        myBikeNetwork.StationsOnNetwork.get(int(trip[3])), 
                        myBikeNetwork.StationsOnNetwork[BikeRiskStation])
                
                # 400 meters were choosen as per Singla study where this is the
                # maximum distance users where willing to change the
                # destination station
                if((distance < 400.0) & (distance > 5.0)):
                    NearStations.append(str(BikeRiskStation))
                    UpcomingIdx.append([i, j])
                    # print(str(BikeRiskStation) + ' replaces: ' + str(int(trip[3])))
                    # print('One near station is: ' + str(int(trip[3])) + ' instead of: ' + str(RiskStation))
                    
    if(NearStations):
        # From all posible solutions, only the station with less bikes will be
        # selected as it is intended to avoid those stations to be empty
        RewardDestinationBikes =\
            [myBikeNetwork.getBikeStationObject(s).availableBikes() for s in NearStations]
        Idx = np.where(RewardDestinationBikes == np.min(RewardDestinationBikes))[0][0]
    
        return NearStations[Idx], UpcomingIdx
    
    else:
        return 0, 0

## Main Function


#### Details:

All functions previosly defined are used in order to find the critical dock stations, the ones that can be replaced are analyzed and then modified from the dayItinerary provided for the current time window.

#### Note:

None

In [ ]:
C_MINUTES_IN_HOUR = 60
C_HOURS_IN_DAY = 24
C_OPTIMAL_LEVEL = 10 # 10 bikes / 20 docks


dayItineraryObject = myBikeNetwork.getDayItinerary(0)
dayItinerary = []
for s in dayItineraryObject:
    dayItinerary.append(s.astype(object))

# for idx, _ in enumerate(dayItinerary):
#     print('Itinerary for Station: ' + str(dayItinerary[idx][0][2]))



myBikeNetwork.resetNetwork()
for time in range(C_MINUTES_IN_HOUR*C_HOURS_IN_DAY):
#     if(time % (C_MINUTES_IN_HOUR) == 0):
    if(time % (int(60 * 0.125)) == 0):
        
        # Get Most Critical Stations
        DockRiskStations, BikeRiskStations = getNetworkRiskStatus(myBikeNetwork)
        # Get the itinerary for the next time delta
        UpcomingInfo = upcomingTrips(time, dayItinerary)

        NearStation = 0
        UpcomingIdx = [0, 0]
        # print('\n\nIt ends ****************')
        # print('********************** It begins \n\n')
        for stationId in BikeRiskStations[:,0]:
            NearStation, UpcomingIdx =\
                retrieveRiskMeasurements(
                    stationId,
                    UpcomingInfo,
                    myBikeNetwork,
                    RouteAnalysisSUMO)
            if(NearStation != 0):
                NearStationObject = myBikeNetwork.getBikeStationObject(
                    int(NearStation))
                ProblemAssesment = C_OPTIMAL_LEVEL - NearStationObject.availableBikes()
                # ProblemAssesment = 1
                
                if((ProblemAssesment >= len(UpcomingIdx)) & (len(UpcomingInfo) > 0)):
                    for i, s in enumerate(UpcomingIdx):
                        # Index for the three required elements of UpcomingInfo
                        # Trip info, dayItineraryTripIndex and dayItineraryIdx
                        idx_1 = s[0]
                        # Index for the trips that are relevant within the UpcomingInfo
                        # element
                        idx_2 = s[1]
                        dayItineraryIdx = UpcomingInfo[idx_1][2]
                        dayItineraryStationIdx = UpcomingInfo[idx_1][1]
                        UpcomingTrips = UpcomingInfo[idx_1][0]
                        
                        # print('Test #1: ' + str(UpcomingInfo[idx_1][0]))
                        # print('Test #2: ' + str(dayItinerary[dayItineraryIdx]))
                        
                        for i, boolIdx in enumerate(dayItineraryStationIdx):
                            if(
                                (boolIdx == True) & \
                                ((dayItinerary[dayItineraryIdx][i][1]) != BSN.C_ARRIVALS_IDX) &\
                                (UpcomingInfo[idx_1][0][idx_2][3] == dayItinerary[dayItineraryIdx][i][3])):
                                
                                print(
                                    'Time %.2f' % (time/60) + ': ' +
                                    'Station ' + str(int(dayItinerary[dayItineraryIdx][i][3])) +\
                                    ' is replaced by station: ' + str(NearStation))
                                dayItinerary[dayItineraryIdx][i][3] = float(NearStation)
                                
                                
                        # print('Test #3: ' + str(dayItinerary[dayItineraryIdx])
                    
    
    # Update the Stations based on the dayItinerary provided / modified
    updateStatusOfStations(time,dayItinerary)
    
#     if(time % C_MINUTES_IN_HOUR == 0):
#         print('At time: ' + str(time) + ' minutes.') 
#         print('Dock Risk Stations: ')
#         print(DockRiskStations)
#         print('Bike Risk Stations: ')
#         print(BikeRiskStations)
#         print()


In [ ]:
#Relocation events on every station
print([ (s, myBikeNetwork.getBikeStationObject(s).getStationStats()[0]) for s in myBikeNetwork.BikeStationsDict.keys() ])

In [ ]:
C_BIKESTATION = 2

currentBikeStation =\
    myBikeNetwork.getBikeStationObject(C_BIKESTATION)
print(currentBikeStation)

In [ ]:
destinations = \
    list(
    currentBikeStation\
    .stationCDFDepartures\
    .wkDayDepartures[0]\
    .Destino_Id)
hist, bin_edges = \
    np.histogram(
        destinations, bins=np.max(destinations),
        density=True)

hist = np.append(hist, [0.0])
plt.figure(1)
plt.title('Station ID Destination of trips from station: ' + str(C_BIKESTATION))
plt.plot(bin_edges, hist)
plt.figure(2)
plt.title('CDF of Station ID Destination of trips from station: ' + str(C_BIKESTATION))
plt.plot(bin_edges, BSCDF.computeCDF(hist,bin_edges))



In [ ]:
C_NUMBER_OF_TRIPS = 50
C_WEEKDAY = 5

Arrivals = np.empty(0)
Departures = np.empty(0)
for i in range(C_NUMBER_OF_TRIPS):
    BikeStationTrips = \
        currentBikeStation.getTripsOnWeekday(C_WEEKDAY)

    tArrivals =\
        BikeStationTrips[
            BikeStationTrips[:, 1] ==
            BSN.C_ARRIVALS_IDX]
    Arrivals = np.r_[Arrivals, tArrivals[:,0]]


    tDepartures =\
        BikeStationTrips[
            BikeStationTrips[:, 1] == 
            BSN.C_DEPARTURES_IDX]
    Departures = np.r_[Departures, tDepartures[:,0]]


In [ ]:
BikeStationTrips

In [ ]:
plt.figure(1)
plt.title('Arrivals Station: ' + 
    currentBikeStation.getStationId())
plt.hist(Arrivals)
print(np.mean(Arrivals))

plt.figure(2)
plt.title(
    'Departures Station: ' + 
    currentBikeStation.getStationId())
plt.hist(Departures)
print(np.mean(Departures))

In [ ]:
ahist, abin_edges = \
    np.histogram(
        Arrivals, bins=24 * 8,
        density=True,
        range=(0.0, 24.0))
ahist = np.append(ahist, [0.0])

In [ ]:
dhist, dbin_edges = \
    np.histogram(
        Departures, bins=24 * 8,
        density=True,
        range=(0.0, 24.0))
dhist = np.append(dhist, [0.0])

In [ ]:
x =\
currentBikeStation.stationCDFArrivals\
.wkDayDepartures_cdf[C_WEEKDAY][0]

y =\
currentBikeStation.stationCDFArrivals\
.wkDayDepartures_cdf[C_WEEKDAY][1]

acdf = BSCDF.computeCDF(
            ahist,abin_edges)

plt.plot(x/3600,y)
plt.plot(24.0*np.arange(len(acdf))/len(acdf), acdf)

In [ ]:
x =\
currentBikeStation.stationCDFDepartures\
.wkDayDepartures_cdf[C_WEEKDAY][0]

y =\
currentBikeStation.stationCDFDepartures\
.wkDayDepartures_cdf[C_WEEKDAY][1]

dcdf = BSCDF.computeCDF(
            dhist,dbin_edges)

plt.plot(x/3600,y)
plt.plot(24.0*np.arange(len(dcdf))/len(dcdf), dcdf)